In [2]:
import sys, os, re

sys.path = (["../../src/", "../../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path

import qubx

%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.interfaces import TriggerEvent, PositionsTracker, IStrategyContext

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers.riskctrl import AtrRiskTracker, StopTakePositionTracker
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer, FixedLeverageSizer
from qubx.trackers.composite import CompositeTrackerPerSide
from qubx.data.readers import CsvStorageDataReader, AsPandasFrame
from qubx.data.helpers import loader

from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet, chart_signals

# - - - - - - - - - - - - - - - - - - - - - - - -
from tests.qubx.trackers.trackers_gathering_test import GuineaPig


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [ ]:
r = CsvStorageDataReader(this_project_root() / "tests/data/csv_1h") # type: ignore  # noqa: F821
I = lookup.find_symbol("BINANCE.UM", "BTCUSDT")  # noqa: E741

In [8]:
class ComplexCompositeTest(GuineaPig):
    def tracker(self, ctx: IStrategyContext) -> PositionsTracker:
        sizer, timeframe = FixedLeverageSizer(1.0), "1h"
        return CompositeTrackerPerSide(
            long_trackers=[
                AtrRiskTracker(
                    take_target=5, stop_risk=3, atr_timeframe=timeframe, atr_period=5, sizer=sizer, risk_controlling_side="broker",
                ),
                StopTakePositionTracker(stop_risk=10, sizer=sizer),
            ],
            short_trackers=[
                AtrRiskTracker(
                    take_target=5, stop_risk=3, atr_timeframe=timeframe, atr_period=5, sizer=sizer, risk_controlling_side="broker",
                ),
                StopTakePositionTracker(stop_risk=10, sizer=sizer),
            ],
        )

In [7]:
assert I is not None

rep = simulate(
    strategies={
        "Composited": (
            strategy := ComplexCompositeTest(
                tests={
                    "2023-07-05 00:00:00": I.signal(-1),
                }
            )
        )
    },
    data={"ohlc(1h)": r}, capital=10000, instruments=["BINANCE.UM:BTCUSDT"], commissions="vip0_usdt", debug="DEBUG",
    start="2023-07-01", stop="2023-08-01",
)


2023-07-01 00:00:00.000 [🐞] [simulator] :: Preparing simulated trading on BINANCE.UM for 10000 USDT...
2023-07-01 00:00:00.000 [ℹ️] SimulatedDataProvider.binance.um is initialized
2023-07-01 00:00:00.000 [🐞] [simulator] :: Setting default schedule: 0 */1 * * *


Simulating:   0%|          | 0/100 [00:00<?, ?%/s]

2023-07-01 00:00:00.000 [ℹ️] SimulatedDataProvider ::: Simulation started at 2023-07-01 00:00:00 :::
2023-07-01 00:00:00.000 [🐞]   [IteratedDataStreamsSlicer] :: Preloading initial data for ohlc.1Min 2023-07-01 00:00:00 : 2023-08-01 00:00:00 ...
2023-07-01 00:00:05.000 [🐞] [ProcessingManager] :: Invoking ComplexCompositeTest on_fit
2023-07-01 00:00:05.000 [🐞] [ProcessingManager] :: ComplexCompositeTest is fitted
2023-07-05 00:00:00.000 [🐞] [BrokerSideRiskController(BINANCE.UM:FUTURE:BTCUSDT)] :: New signal: Target +0.000000 for ComplexCompositeTest 30756.10 +0.000000 BINANCE.UM:FUTURE:BTCUSDT (Closing opposite signal -1 )
2023-07-05 00:00:00.000 [🐞] [BrokerSideRiskController(BINANCE.UM:FUTURE:BTCUSDT)] :: New signal: Target +0.000000 for ComplexCompositeTest 30756.10 +0.000000 BINANCE.UM:FUTURE:BTCUSDT (Closing opposite signal -1 )
2023-07-05 00:00:00.000 [🐞] [BrokerSideRiskController(BINANCE.UM:FUTURE:BTCUSDT)] :: New signal: Target -0.325000 for ComplexCompositeTest 30756.10 -1.00000

InvalidOrder: Order SIM-ORDER-BTCUSDT-100004 not found for BTCUSDT